In [1]:
# Import the libraries
import pandas as pd
from IPython.display import display, HTML
# Import the libraries
import os, json, ast
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import textwrap

# Set the display width to control the output width
pd.set_option('display.width', 100)
# Read the dataset and read the Laptop Dataset
df = pd.read_excel('ConsolidatedTravel.xlsx')
df

,City,Best TimeToVisit,City_desc,CountofPlaces,CostofStay,Summary
0,Manali,October-June,[' One of the most popular hill stations in Hi...,50,Hotel : 500 -5000 GuestHouse : 2000 -8000 ...,City : Manali ;; Best TimeToVisit : October-...
1,Leh Ladakh,July-October,"["" Ladakh is a union territory in the Kashmir ...",52,Hotel : 1000 - 4000 GuestHouse : 800 - 3000 ...,City : Leh Ladakh ;; Best TimeToVisit : July...
2,Coorg,September-June,[' Located amidst imposing mountains in Karnat...,34,Hotel : 2000 -8000 GuestHouse : 1000 - 4000...,City : Coorg ;; Best TimeToVisit : September...
3,Andaman,October-March,[' Replete with turquoise blue water beaches a...,52,Hotel : 500 - 10000 GuestHouse : 500 - 6000 ...,City : Andaman ;; Best TimeToVisit : October...
4,Lakshadweep,September-February,"["" Formerly known as Laccadive Islands, Laksha...",15,Hotel : 5000 -20000 GuestHouse : 3000 - 2000...,City : Lakshadweep ;; Best TimeToVisit : Sep...
...,...,...,...,...,...,...
95,Pushkar,October-March,[' Pushka is a small temple town located aroun...,20,Hotel : 2000 -8000 GuestHouse : 1000 - 4000 ...,City : Pushkar ;; Best TimeToVisit : October...
96,Chittorgarh,October-March,"[' Located in South-Eastern Rajasthan, Chittor...",19,Hotel : 2000 -8000 GuestHouse : 1000 - 4000 ...,City : Chittorgarh ;; Best TimeToVisit : Oct...
97,Nahan,September-November,"[' Away from the buzzing crowd of the city, Na...",20,Hotel : 2000 -8000 GuestHouse : 1000 - 4000 ...,City : Nahan ;; Best TimeToVisit : September...
98,Lavasa,October-March,"["" Known as India's newest hill station, the L...",5,Hotel : 500 - 6000 GuestHouse : 800 - 3000 ...,City : Lavasa ;; Best TimeToVisit : October-...


In [2]:
# Read the OpenAI API key
openai.api_key = open("OPENAI_API_Key.txt", "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [3]:
df['Summary'][0]

'City : Manali  ;;  Best TimeToVisit : October-June  ;;  City_desc : [\' One of the most popular hill stations in Himachal, Manali offers the most magnificent views of the Pir Panjal and the Dhauladhar ranges covered with snow for most parts of the year. \', \'\', " With the Covid-19 pandemic going on, Manali has evolved to be a place loved by young people looking for longer stays for workation. With ambient cafes, good wifi availability, small eateries and convenient shops, Old Manali is among the favorite neighbourhood for such people. There are many homestays and hostels offering dorm beds for cheap for longer durations. Manali has many trekking options around it, making it a great base for exploring this side of Himalayas. River Beas provides great rafting options in the nearby town of Kullu. Adjoining Parvati river, lies the Parvati Valley with  Kasol, Manikaran, Tosh and small villages attracting travellers for longer stays. Atal Tunnel now allows travellers to reach Sissu within

In [66]:
def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''

    delimiter = "####"

    example_user_dict = {'Location': "Hills",
                        'Activities':"Trekking",
                        'TimeOfYear': "Jan-Mar",
                        'Climate': "Cold",
                        'Group': "couple",
                        'RoomBudget': "2000"}

    example_user_req = {'Location': "_",
                        'Activities': "_",
                        'TimeofYear': "_",
                        'Climate': "_",
                        'Group': "_",
                        'RoomBudget': "_"}

    system_message = f"""
    You are a intelligent travel advisor and recommend the best travel destination based on user preferences. 
    You need to ask relevant questions and understand the user profile by analysing the user's responses.

    Begin with asking his name. This is very important and you will be addressing him on this

    To make an informed suggestion, ask the user for the following key details
    {delimiter}
    Preferred Destination Type – Do they prefer beaches, hills, forests, islands, or deserts?
    Activities of Interest – Are they looking for adventure (trekking, water sports), cultural experiences (heritage sites, temples), or relaxation (resorts, spas)?
    Budget – What type of room are they looking for {delimiter} present the room details along with budget for them to choose){delimiter}
    Climate Preference – Do they prefer cold, moderate, or tropical weather? {delimiter} present examples for clear response {delimiter}
    Travel Timeframe – What is their preferred season of travel (season or specific months)?
    Group Type – Are they traveling solo, as a couple, with family, or in a group?
    Cultural Interests – Do they prefer places rich in history, local culture, and traditions?

    {delimiter}

    VERY IMPORTANT. If the value for any key is '_' please ensure to ask the question. You have to capture all the values before recommending to the user. 
    {delimiter}

    For each of the question, please few examples so that you can guide the  user for better options. Otherwise you will be penalized

    The python dictionary looks like this
    {{'Destination': 'values','Activities': 'values','TimeofYear': 'values','Climate': 'values','Group': 'values','RoomBudget': 'values'}}
    The value for 'Budget' should be a numerical value extracted from the user's response.
    
    {delimiter}

    Based on their answers, recommend the best destination from the available options, providing reasons why it fits their needs. 
    Also, suggest must-visit attractions and activities they can explore during their trip.
    
    {delimiter}

    Please reask the question in a differnet way if any of the response is inaccurate or not as expectation (IMPORTANT)

    {delimiter}

    Key Values that you need to capture based on the discussion with user:
    Location – ["The specific location recommended"]
    Activities –  ["Adventure, cultural experiences, relaxation, sightseeing, etc"]
    TimeofYear – ["During whuch months is the traveller planning"]
    Climate – ["The Climate the user wants to explore"]
    Group - ["Whether the user is planning to travel Solo, Couple or Family etc]
    RoomBudget - ["The budget they have for room"]



    - Do not randomly assign values to any of the keys.
    - The values need to be inferred from the user's response.

    {delimiter}

    Please ask the question again if any of the 6 keys ('Location','Activities','TimeofYear','Climate','Group','RoomBudget') does not have a value. It cannot be "_" or spaces etc.
    It has to be a valid values.    otherwise you will be penalized
    
    {delimiter}

    To fill the dictionary, you need to have the following chain of thoughts:
    Follow the chain-of-thoughts below and only output the final updated python dictionary for the keys as described in {example_user_req}. \n


    Thought 2: Now, you are trying to fill the values for the rest of the keys which you couldn't in the previous step.
    Remember the instructions around the values for the different keys.
    Ask questions you might have for all the keys to strengthen your understanding of the user's profile.
    If yes, move to the next Thought. If no, ask question on the keys whose values you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the key you want to understand value for.
    {delimiter}

    {delimiter}
    Thought 3: Check if you have correctly updated the values for the different keys in the python dictionary.
    If you are not confident about any of the values, ask clarifying questions.
    {delimiter}


    Here is a sample conversation between the user and assistant:

    Assistant: " Hello! I’d love to help you find the perfect travel destination. To personalize my recommendation, I need a few details from you.
    
                What type of location do you prefer? (Beach, Hills, Forest, Desert, Island)
                What kind of activities are you interested in? (Adventure, Culture, Relaxation, Sightseeing)
                What is your approximate budget for this trip?
                Do you have any climate preferences? (Cold, Warm, Tropical, Moderate)
                When are you planning to travel? (Specific months or seasons)
                Are you traveling solo, as a couple, with family, or with friends?
                
    Let me know, and I’ll suggest the best travel destination for you!  "



    User: "I’m looking for a beach destination with a mix of adventure and relaxation. My budget is around $1000. 
I prefer a tropical climate and plan to travel in December."

Remember you need not NOT ask all questions at once and go with 1-2 questions initially and then based on that ask further questions. You will be penalized if 
you do not follow this.


List of all travel locations along with budget, time of visit, description of city along with experience can be found here. 
{delimiter} {

df['Summary']}   

{delimiter} 
Please refer this also when you are interacting with the user


    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]
    # conversation = system_message
    return conversation

In [5]:
# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff, starting at 1 second and maxing out at 20 seconds delay
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_completions(input, json_format = False):
    MODEL = 'gpt-3.5-turbo'

    system_message_json_output = """<<. Return output in JSON format to the key output.>>"""

    # If the output is required to be in JSON format
    if json_format == True:
        # Append the input prompt to include JSON response as specified by OpenAI
        input[0]['content'] += system_message_json_output
        # JSON return type specified
        chat_completion_json = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            response_format = { "type": "json_object"},
            seed = 1234)

        output = json.loads(chat_completion_json.choices[0].message.content)

    # No JSON return type specified
    else:
        chat_completion = openai.chat.completions.create(
            model = MODEL,
            messages = input,
            seed = 2345)

        output = chat_completion.choices[0].message.content

    return output

In [6]:
# Define a function called moderation_check that takes user_input as a parameter.

def moderation_check(user_input):
    # Call the OpenAI API to perform moderation on the user's input.
    response = openai.moderations.create(input=user_input)

    # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"

In [7]:
def intent_confirmation_layer(response_assistant):

    delimiter = "####"

    prompt = f"""
    You are a senior evaluator who has an eye for detail.The input text will contain a user requirement captured through 6 keys.
    You are provided an input. You need to evaluate if the input text has the following keys:



                       {{'Location': 'values',
                        'Activities': 'values',
                        'TimeofYear': 'values',
                        'Climate': 'values',
                        'Group': 'values',
                        'RoomBudget': 'values'}}
                        
    The 'RoomBudget' key can take only a numerical value.
    Next you need to evaluate if the keys have the the values filled correctly.

    If any of the values are '_' please outout 'No', else you must output 'Yes'. You will be penalized if you do not follow this.
    
    Thought 1 - Output a string 'Yes' if the values are correctly filled for all keys, otherwise output 'No'.
    THought 2 - Think carefully before the answering.


    Only output a one-word string in JSON format at the key 'result' - Yes/No.


    Please check if all the 6 keys ('Location','Activities','TimeofYear','Climate','Group','RoomBudget') has a  value before responding, 
    otherwise you will be penalized. Your response will only be a 'Yes' or 'No'
    


    Also none of the keys should have a value of "_" or spaces. You need to check all of this before confirming.
    
        """
    messages=[{"role": "system", "content":prompt },
              {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = messages,
                                    response_format={ "type": "json_object" },
                                    seed = 1234
                                    # n = 5
                                    )

    json_output = json.loads(response.choices[0].message.content)

    return json_output

In [8]:
def dictionary_present(response):
    delimiter = "####"

    user_req=          {'Location': ['hills'],
                        'Activities': ['trekking','swimming'],
                        'TimeofYear': ['January','February'],
                        'Climate': ['Winter','spring'],
                        'Group': ['family','couple'],
                        'RoomBudget': [2000,3000]}

    prompt = f"""You are a python expert. You are provided an input.
            You have to check if there is a python dictionary present in the string.
            It will have the following format {user_req}.
            Your task is to just extract the relevant values from the input and return only the python dictionary in JSON format.
            The output should match the format as {user_req}.

            {delimiter}
            Make sure that the value of budget is also present in the user input. ###
            The output should contain the exact keys and values as present in the input.
            Ensure the keys and values are in the given format:


            {{'Location': ['hills'],
                        'Activities': ['trekking','swimming'],
                        'TimeofYear': ['January','February'],
                        'Climate': ['Winter'],
                        'Group': ['family'],
                        'RoomBudget': [2000]}}


            {delimiter}

            """
    messages = [{"role": "system", "content":prompt },
                {"role": "user", "content":f"""Here is the user input: {response}""" }]

    confirmation = get_chat_completions(messages, json_format = True)

    return confirmation

In [9]:
# def Top_Three_Suggestions(recommendations):

#     delimiter = "####"

#     prompt = f"""
#     You are a senior evaluator who has can identity top 3 destinations based on the user's expections. 
    

#     {delimiter}

#     The details of all the locations, budget, places to visit etc can be found at {df['Summary']}
    
#     {delimiter}

#     Your job is to identity the top 3 locations and rank them according to choice and present to the table.


#     ###
#     You will be returning a python dictionary

#    {delimiter}


# {
#   "recommendations": [
#     {
#       "rank": 1,
#       "city": "Andaman & Nicobar Islands",
#       "budget": "$800 - $1200",
#       "activities": ["Scuba Diving", "Snorkeling", "Relaxation", "Beach Walks"],
#       "climate": "Tropical",
#       "recommendedMonth": "December - March",
#       "description": "Andaman & Nicobar Islands offer crystal-clear waters, white sandy beaches, and thrilling water sports. Havelock and Neil Islands are perfect for diving and snorkeling, while Radhanagar Beach is a serene retreat for relaxation."
#     },
#     {
#       "rank": 2,
#       "city": "Goa",
#       "budget": "$600 - $1000",
#       "activities": ["Nightlife", "Water Sports", "Sightseeing", "Beach Relaxation"],
#       "climate": "Tropical",
#       "recommendedMonth": "November - February",
#       "description": "Goa is a vibrant beach destination known for its lively nightlife, water adventures, and stunning Portuguese heritage. Enjoy beach parties, scenic forts, and laid-back shacks offering delicious seafood."
#     },
#     {
#       "rank": 3,
#       "city": "Bali",
#       "budget": "$1000 - $1500",
#       "activities": ["Temple Visits", "Surfing", "Hiking", "Cultural Experiences"],
#       "climate": "Tropical",
#       "recommendedMonth": "April - October",
#       "description": "Bali is a paradise for travelers seeking a mix of culture, adventure, and relaxation. Visit iconic temples, surf at Kuta Beach, explore lush rice terraces, and enjoy traditional Balinese hospitality."
#     }
#   ]
# }
#         Explanation of Fields:
#         rank → The ranking of the destination based on user preferences.
#         city → The recommended destination.
#         budget → The estimated travel budget for the location.
#         activities → A list of key activities available at the destination.
#         climate → The general climate type of the location.
#         recommendedMonth → The best time to visit the destination.
#         description → A brief write-up explaining why the place is a great fit for the traveler.


# {delimiter}

# Please ensure to review the user expectations against the Summary. 

    
#     """

#     messages=[{"role": "system", "content":prompt },
#               {"role": "user", "content":f"""Here is the input: {response_assistant}""" }]

#     response = openai.chat.completions.create(
#                                     model="gpt-3.5-turbo",
#                                     messages = messages,
#  #                                   response_format={ "type": "json_object" },
#                                     seed = 1234
#                                     # n = 5
#                                     )

#     output = response.choices[0].message.content

#     return output

In [10]:
# def Top_Three_Suggestions(recommendations):

#     delimiter = "####"

#     prompt = f"""
#     You are a senior evaluator who has can identity top 3 destinations based on the user's expections. 
    

#     {delimiter}

#     The details of all the locations, budget, places to visit etc can be found at {df['Summary']}
    
#     {delimiter}

#     Your job is to identity the top 3 locations and rank them according to choice and present to the table.


#     ###
#     You will be returning a string in the sample format given below

#    {delimiter}

# EXAMPLE format:::
#    {delimiter}



# {{
#     "Rank": [1, 2, 3],
#     "City": ["Andaman & Nicobar Islands", "Goa", "Bali"],
#     "Budget": ["$800 - $1200", "$600 - $1000", "$1000 - $1500"],
#     "Activities": [
#         "Scuba Diving, Snorkeling, Relaxation, Beach Walks",
#         "Nightlife, Water Sports, Sightseeing, Beach Relaxation",
#         "Temple Visits, Surfing, Hiking, Cultural Experiences"
#     ],
#     "Climate": ["Tropical", "Tropical", "Tropical"],
#     "RecommendedMonth": ["December - March", "November - February", "April - October"],
#     "Description": [
#         "Andaman offers crystal-clear waters, white sandy beaches, and thrilling water sports. Havelock and Neil Islands are perfect for diving and snorkeling, while Radhanagar Beach is a serene retreat.",
#         "Goa is known for its lively nightlife, water adventures, and Portuguese heritage. Enjoy beach parties, scenic forts, and shacks offering delicious seafood.",
#         "Bali offers a mix of culture, adventure, and relaxation. Visit iconic temples, surf at Kuta Beach, explore lush rice terraces, and experience Balinese hospitality."
#     ]
# }}

# {delimiter}


# {delimiter}

# LAYOUT
# {{
#     "Rank": [1, 2, 3],  
#     "City": ["{City_Name}", "{City_Name}", "{City_Name}"],  
#     "Budget": ["{Budget_Range}", "{Budget_Range}", "{Budget_Range}"],  
#     "Activities": [
#         "{Activity_1, Activity_2, ...}", 
#         "{Activity_1, Activity_2, ...}", 
#         "{Activity_1, Activity_2, ...}"
#     ],  # List of key activities
#     "Climate": ["{Climate_Type}", "{Climate_Type}", "{Climate_Type}"],  
#     "RecommendedMonth": ["{Best_Time_to_Visit}", "{Best_Time_to_Visit}", "{Best_Time_to_Visit}"],  
#     "Description": [
#         "{ write-up}",
#         "{ write-up}",
#         "{ write-up}"
#     ]  
# }}

# Please ensure to review the user expectations against the Summary. 

    
#     """

#     messages=[{"role": "system", "content":prompt },
#               {"role": "user", "content":f"""Here is the input: {recommendations}""" }]
#     print ("getting top 3")
#     response = openai.chat.completions.create(
#                                     model="gpt-3.5-turbo",
#                                     messages = messages,
#  #                                   response_format={ "type": "json_object" },
#                                     seed = 1234
#                                     # n = 5
#                                     )

#     json_output = json.loads(response.choices[0].message.content)

#     return json_output

In [65]:
def Top_Three_Suggestions(conversation,recommendations):

    delimiter = "####"
    tabbled = '  '

    prompt = f"""
    
    You are a professional travel agent. Your task is to create a personalized **travel proposal** 
    recommending the **top 3 best destinations** based on the user's preferences in {recommendations}. 
    The destinations should be ranked from **most recommended to least recommended** based on their fit with the user's needs.  

You need to provide a detailed report that can be presented to the Customer. Review the entre Chat along with {recommendations} when you prepare the content.

#Return the DETAILED proposal in structured text format so it can be formatted into a **PDF** (Atleast 300 words)

Details of the Place, Activities, Budget, Climate should all be present. Please have a customized message to the user and address him by his name

YOU WILL BE PENALIZED IF YOU DO NOT DO THIS
    
           
    
    """
    
    # messages=[{"role": "system", "content":conversation },
    #           {"role": "user", "content":f"""Here is the input: {prompt}""" }]

    conversation.append({"role": "user", "content": str({prompt})})

#    print (conversation)
#    print (messages)
    print ("message above")
    response = openai.chat.completions.create(
                                    model="gpt-3.5-turbo",
                                    messages = conversation,
 #                                   response_format={ "type": "json_object" },
                                    seed = 1234
                                    # n = 5
                                    )
    print ("in output")
   # output = response.choices[0].message.content

    return response
    # json_output = json.loads(response.choices[0].message.content)

    # return json_output

In [12]:
def dialogue_mgmt_system():
    
    global user_criteria_built
    global recommendations
    global conversation
    
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    conversation.append({"role": "assistant", "content": introduction})

    display(introduction + '\n')


    user_input = ''

    while(user_input != "exit"  and user_criteria_built=="N"):

        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if user_criteria_built == "N":

            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_completions(conversation)
            
            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break


            confirmation = intent_confirmation_layer(response_assistant)

            print("Intent Confirmation Yes/No:",confirmation.get('result'))

            if "No" in confirmation.get('result'):
                conversation.append({"role": "assistant", "content": str(response_assistant)})
                print("\n" + str(response_assistant) + "\n")

            else:
                #print("\n" + str(response_assistant) + "\n")
                print('\n' + "Variables extracted!" + '\n')
                conversation.append({"role": "assistant", "content": str(response_assistant)})

                user_criteria_built = "Y"
                recommendations = dictionary_present(response_assistant)
                
                print("\n" + str(recommendations) + "\n")
                print ("coming out of loop")


    
    # with open("output.txt", "w", encoding="utf-8") as file:
    #     file.write(output_recommendations)

In [76]:


user_criteria_built = "N"
recommendations = ""
conversation =[]

dialogue_mgmt_system()
output_recommendations = ""


if (user_criteria_built =="Y"):
    print ("going to fetch top 3 locations")
    output_recommendations = Top_Three_Suggestions(conversation,recommendations)
    print (output_recommendations)


    
    x, y = 25, 750 
    
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    c.setFont("Helvetica", 8)
    
    for line in output_recommendations.choices[0].message.content.splitlines():
        if (line == "") or (line == "\n"):
            continue
        else:
    
            if len(line)> 100:
                wrapped_lines = textwrap.wrap(line, width=150)
                for part in wrapped_lines:
                    c.drawString(x, y, part.strip())
                    y -= 20
    
            else:
                c.drawString(x, y, line.strip())
                y -= 20  # Move down for the next line
    
    c.save()
    

'Hello! I’d love to help you find the perfect travel destination. To personalize my recommendation, I need a few details from you.\n\nWhat type of location do you prefer? Beach, Hills, Forest, Desert, or Island?\n'

 forest and hills


Intent Confirmation Yes/No: No

Great choice! 

What kind of activities are you interested in? Adventure (like trekking, wildlife safari), Cultural experiences (like exploring heritage sites, temples), or Relaxation (like staying at resorts, spas)?



 wildlife safari, kayaking, rafting, water activities and sightseeing


Intent Confirmation Yes/No: No

Awesome! 

What is your approximate budget for this trip? You can choose from the following room budget options:
1. Budget Room ($50-$100 per night)
2. Standard Room ($100-$200 per night)
3. Deluxe Room ($200-$300 per night)
4. Premium Room ($300+ per night)



...

Intent Confirmation Yes/No: No

When are you planning to travel? Are you looking to travel during specific months or seasons?



 May


Intent Confirmation Yes/No: No

Are you planning to travel solo, as a couple, with family, or in a group?



 solo


Intent Confirmation Yes/No: Yes

Variables extracted!


{'Location': ['Forest', 'Hills'], 'Activities': ['Wildlife safari', 'kayaking', 'rafting', 'water activities', 'sightseeing'], 'TimeofYear': ['May'], 'Climate': ['Tropical spring'], 'Group': ['Solo'], 'RoomBudget': ['Deluxe Room ($200-$300 per night)']}

coming out of loop
going to fetch top 3 locations
message above
in output
ChatCompletion(id='chatcmpl-BBkL8bwyDMhF2kG3CpF0IC0TuF8GC', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="### Travel Proposal for Solo Traveler in May seeking Forest and Hills Destination with Adventure Activities\n\nDear Solo Traveler,\n\nBased on your preferences for a destination that offers a mix of forest and hills with adventure activities like wildlife safari, kayaking, rafting, water activities, and sightseeing, I have curated a list of the top 3 recommended destinations tailored just for you. These destinations match your requirements in terms o

In [70]:
output_recommendations

ChatCompletion(id='chatcmpl-BBhQpMYjF3zbGi4cT3VfqVdmZMgUJ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"Location": \'_\', \'Activities\': \'_\', \'TimeofYear\': \'_\', \'Climate\': \'Tropical\', \'Group\': \'_\', \'RoomBudget\': \'Medium\'}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1742128259, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=33, prompt_tokens=1765, total_tokens=1798, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [72]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import textwrap

if (user_criteria_built =="Y"):
    print ("going to fetch top 3 locations")
    output_recommendations = Top_Three_Suggestions(conversation,recommendations)
    print (output_recommendations)

x, y = 25, 750 

c = canvas.Canvas(pdf_filename, pagesize=letter)
c.setFont("Helvetica", 8)

for line in output_recommendations.choices[0].message.content.splitlines():
    if (line == "") or (line == "\n"):
        continue
    else:

        if len(line)> 100:
            wrapped_lines = textwrap.wrap(line, width=150)
            for part in wrapped_lines:
                c.drawString(x, y, part.strip())
                y -= 20

        else:
            c.drawString(x, y, line.strip())
            y -= 20  # Move down for the next line

c.save()

going to fetch top 3 locations
message above
in output
ChatCompletion(id='chatcmpl-BBhTsldJR4P41nyQBHoYdeAWdkr4G', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I understand. Let's proceed with creating a personalized travel proposal for you based on your preferences for an island destination with adventure and sightseeing activities, a tropical climate, traveling in spring, and a medium budget package.\n\n### Travel Proposal:\n\n1. **Location: Bali, Indonesia**\n   - **Activities:** Bali offers a perfect blend of adventure and sightseeing activities with options like surfing, snorkeling, exploring temples, and visiting scenic rice terraces.\n   - **Climate:** Enjoy a tropical climate with warm temperatures and occasional rain showers during the spring season in Bali.\n   - **Budget:** The medium package budget aligns well with Bali's range of accommodations and activities.\n\n2. **Location: Maldives**\n   - **Activities:** Indulge

In [ ]:
print(debug_conversation[0]['content'])